In [ ]:
# Digital Authenticity Verification System (Non-ML Forensic Engine)
# Jupyter Notebook Prototype (v4 - FULL FORENSIC SYSTEM)
# -----------------------------------------------------
# Multi-class classification + Heatmaps + Risk Score + Report Generator
# Pure Passive Techniques | No ML | No DL | No AI

# ================= SETUP =================
import cv2
import numpy as np
import pywt
from PIL import Image
from scipy.fftpack import fft2, fftshift
from scipy.stats import entropy, normaltest
from scipy.signal import correlate2d
import matplotlib.pyplot as plt
from skimage.filters import sobel
import exifread
import json
from datetime import datetime

# ================= UTILITIES =================
def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError("Image not found")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ================= FAST NOISE =================
def extract_noise_residual(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    denoised = cv2.GaussianBlur(gray, (5,5), 0)
    return gray.astype(np.float32) - denoised.astype(np.float32)

# ================= FORENSIC FEATURES =================
def noise_gaussian_test(residual):
    sample = residual.flatten()[::50]
    _, p = normaltest(sample)
    return p


def noise_isotropy(residual):
    gradx = cv2.Sobel(residual, cv2.CV_64F, 1, 0)
    grady = cv2.Sobel(residual, cv2.CV_64F, 0, 1)
    return np.std(gradx) - np.std(grady)


def noise_autocorrelation(residual):
    small = cv2.resize(residual, (128,128))
    corr = correlate2d(small, small, mode='same')
    center = corr[corr.shape[0]//2, corr.shape[1]//2]
    return center / (np.mean(corr)+1e-8)


def cfa_periodicity(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    even = gray[0::2, 0::2]
    odd = gray[1::2, 1::2]
    return abs(np.mean(even) - np.mean(odd))


def frequency_spectrum(img):
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),(256,256))
    f = fft2(gray)
    mag = np.log(np.abs(fftshift(f))+1)
    return np.mean(mag), np.std(mag)


def diffusion_trace(residual):
    small = cv2.resize(residual,(128,128))
    h,w = small.shape
    left = small[:, :w//2]
    right = np.fliplr(small[:, w//2:])
    return np.mean(np.abs(left - right[:,:left.shape[1]]))


def wavelet_energy(img):
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),(256,256))
    coeffs = pywt.wavedec2(gray,'haar',level=2)
    return sum(np.sum(np.square(b)) for lvl in coeffs[1:] for b in lvl)


def entropy_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist,_ = np.histogram(gray.flatten(),bins=256,density=True)
    return entropy(hist+1e-10)


def edge_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = sobel(gray)
    return np.mean(edges), np.std(edges)


def texture_randomness(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return np.std(gray)


def metadata_score(path):
    try:
        with open(path,'rb') as f:
            tags = exifread.process_file(f)
        return len(tags)
    except:
        return 0

# ================= FEATURE VECTOR =================
def forensic_vector(img,path):
    residual = extract_noise_residual(img)
    fmean,fstd = frequency_spectrum(img)
    emean,estd = edge_score(img)

    return {
        "NOISE_STD": float(np.std(residual)),
        "NOISE_GAUSS_P": float(noise_gaussian_test(residual)),
        "NOISE_ISOTROPY": float(noise_isotropy(residual)),
        "NOISE_AUTOCORR": float(noise_autocorrelation(residual)),
        "CFA_PERIOD": float(cfa_periodicity(img)),
        "FREQ_MEAN": float(fmean),
        "FREQ_STD": float(fstd),
        "DIFFUSION_SYM": float(diffusion_trace(residual)),
        "WAVELET": float(wavelet_energy(img)),
        "ENTROPY": float(entropy_score(img)),
        "EDGE_MEAN": float(emean),
        "EDGE_STD": float(estd),
        "TEXTURE": float(texture_randomness(img)),
        "METADATA": int(metadata_score(path))
    }

# ================= MULTI-CLASS RULE ENGINE =================
def multi_class_engine(v):
    ai = 0
    tamper = 0

    # AI indicators
    if v['NOISE_GAUSS_P'] > 0.05: ai+=1
    if abs(v['NOISE_ISOTROPY']) < 0.5: ai+=1
    if v['CFA_PERIOD'] < 2: ai+=1
    if v['DIFFUSION_SYM'] < 5: ai+=1
    if v['METADATA'] == 0: ai+=1

    # Tamper indicators
    if v['EDGE_STD'] > 0.08: tamper+=1
    if v['FREQ_STD'] > 1.0: tamper+=1
    if v['ENTROPY'] > 5.1: tamper+=1

    # Classification
    if ai>=4 and tamper<=1:
        cls = "AI-GENERATED"
    elif tamper>=3 and ai<=1:
        cls = "TAMPERED"
    elif ai>=2 and tamper>=2:
        cls = "HYBRID"
    else:
        cls = "NATURAL"

    return cls, ai, tamper

# ================= RISK SCORE =================
def fraud_risk_score(ai,tamper):
    score = ai*15 + tamper*12
    return min(score,100)

# ================= HEATMAP =================
def region_heatmap(path, grid=8):
    img = load_image(path)
    h,w,_ = img.shape
    heat = np.zeros((grid,grid))

    for i in range(grid):
        for j in range(grid):
            y1 = int(i*h/grid); y2 = int((i+1)*h/grid)
            x1 = int(j*w/grid); x2 = int((j+1)*w/grid)
            patch = img[y1:y2,x1:x2]
            res = extract_noise_residual(patch)
            heat[i,j] = np.std(res)

    heat = cv2.resize(heat,(w,h))
    return heat

# ================= REPORT GENERATOR =================
def generate_report(path,vector,cls,ai,tamper,risk):
    report = {
        "file": path,
        "timestamp": str(datetime.now()),
        "classification": cls,
        "ai_score": ai,
        "tamper_score": tamper,
        "fraud_risk": risk,
        "forensic_vector": vector
    }

    name = path.split('/')[-1].split('.')[0]
    with open(f"{name}_forensic_report.json","w") as f:
        json.dump(report,f,indent=4)

    return report

# ================= PIPELINE =================
def full_forensic_pipeline(path):
    img = load_image(path)
    vector = forensic_vector(img,path)
    cls, ai, tamper = multi_class_engine(vector)
    risk = fraud_risk_score(ai,tamper)
    heat = region_heatmap(path)
    report = generate_report(path,vector,cls,ai,tamper,risk)
    return vector, cls, ai, tamper, risk, heat, report

# ================= VISUAL =================
def visualize_all(path, heat):
    img = load_image(path)
    plt.figure(figsize=(12,4))

    plt.subplot(1,3,1)
    plt.title('Original')
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.title('AI/Tamper Heatmap')
    plt.imshow(heat, cmap='hot')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.title('Overlay')
    plt.imshow(img)
    plt.imshow(heat, cmap='hot', alpha=0.4)
    plt.axis('off')

    plt.show()

print("FULL FORENSIC SYSTEM READY")


In [ ]:
path = "data/Gemini_Generated_Image_ghhom5ghhom5ghho (1).jpg"

vector, cls, ai, tamper, risk, heat, report = full_forensic_pipeline(path)

print("Classification:", cls)
print("AI Score:", ai)
print("Tamper Score:", tamper)
print("Fraud Risk:", risk)
print("Report File Generated")

visualize_all(path, heat)


heatmap

In [ ]:
# Digital Authenticity Verification System (Non-ML Forensic Engine)
# Jupyter Notebook Prototype (v4 - FULL FORENSIC SYSTEM)
# -----------------------------------------------------
# Multi-class classification + Heatmaps + Risk Score + Report Generator
# Pure Passive Techniques | No ML | No DL | No AI

# ================= SETUP =================
import cv2
import numpy as np
import pywt
from PIL import Image
from scipy.fftpack import fft2, fftshift
from scipy.stats import entropy, normaltest
from scipy.signal import correlate2d
import matplotlib.pyplot as plt
from skimage.filters import sobel
import exifread
import json
from datetime import datetime

# ================= UTILITIES =================
def load_image(path):
    img = cv2.imread(path)
    if img is None:
        raise ValueError("Image not found")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# ================= FAST NOISE =================
def extract_noise_residual(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    denoised = cv2.GaussianBlur(gray, (5,5), 0)
    return gray.astype(np.float32) - denoised.astype(np.float32)

# ================= FORENSIC FEATURES =================
def noise_gaussian_test(residual):
    sample = residual.flatten()[::50]
    _, p = normaltest(sample)
    return p


def noise_isotropy(residual):
    gradx = cv2.Sobel(residual, cv2.CV_64F, 1, 0)
    grady = cv2.Sobel(residual, cv2.CV_64F, 0, 1)
    return np.std(gradx) - np.std(grady)


def noise_autocorrelation(residual):
    small = cv2.resize(residual, (128,128))
    corr = correlate2d(small, small, mode='same')
    center = corr[corr.shape[0]//2, corr.shape[1]//2]
    return center / (np.mean(corr)+1e-8)


def cfa_periodicity(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    even = gray[0::2, 0::2]
    odd = gray[1::2, 1::2]
    return abs(np.mean(even) - np.mean(odd))


def frequency_spectrum(img):
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),(256,256))
    f = fft2(gray)
    mag = np.log(np.abs(fftshift(f))+1)
    return np.mean(mag), np.std(mag)


def diffusion_trace(residual):
    small = cv2.resize(residual,(128,128))
    h,w = small.shape
    left = small[:, :w//2]
    right = np.fliplr(small[:, w//2:])
    return np.mean(np.abs(left - right[:,:left.shape[1]]))


def wavelet_energy(img):
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),(256,256))
    coeffs = pywt.wavedec2(gray,'haar',level=2)
    return sum(np.sum(np.square(b)) for lvl in coeffs[1:] for b in lvl)


def entropy_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    hist,_ = np.histogram(gray.flatten(),bins=256,density=True)
    return entropy(hist+1e-10)


def edge_score(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = sobel(gray)
    return np.mean(edges), np.std(edges)


def texture_randomness(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return np.std(gray)


def metadata_score(path):
    try:
        with open(path,'rb') as f:
            tags = exifread.process_file(f)
        return len(tags)
    except:
        return 0

# ================= FEATURE VECTOR =================
def forensic_vector(img,path):
    residual = extract_noise_residual(img)
    fmean,fstd = frequency_spectrum(img)
    emean,estd = edge_score(img)

    return {
        "NOISE_STD": float(np.std(residual)),
        "NOISE_GAUSS_P": float(noise_gaussian_test(residual)),
        "NOISE_ISOTROPY": float(noise_isotropy(residual)),
        "NOISE_AUTOCORR": float(noise_autocorrelation(residual)),
        "CFA_PERIOD": float(cfa_periodicity(img)),
        "FREQ_MEAN": float(fmean),
        "FREQ_STD": float(fstd),
        "DIFFUSION_SYM": float(diffusion_trace(residual)),
        "WAVELET": float(wavelet_energy(img)),
        "ENTROPY": float(entropy_score(img)),
        "EDGE_MEAN": float(emean),
        "EDGE_STD": float(estd),
        "TEXTURE": float(texture_randomness(img)),
        "METADATA": int(metadata_score(path))
    }

# ================= MULTI-CLASS RULE ENGINE =================
def multi_class_engine(v):
    ai = 0
    tamper = 0

    # AI indicators
    if v['NOISE_GAUSS_P'] > 0.05: ai+=1
    if abs(v['NOISE_ISOTROPY']) < 0.5: ai+=1
    if v['CFA_PERIOD'] < 2: ai+=1
    if v['DIFFUSION_SYM'] < 5: ai+=1
    if v['METADATA'] == 0: ai+=1

    # Tamper indicators
    if v['EDGE_STD'] > 0.08: tamper+=1
    if v['FREQ_STD'] > 1.0: tamper+=1
    if v['ENTROPY'] > 5.1: tamper+=1

    # Classification
    if ai>=4 and tamper<=1:
        cls = "AI-GENERATED"
    elif tamper>=3 and ai<=1:
        cls = "TAMPERED"
    elif ai>=2 and tamper>=2:
        cls = "HYBRID"
    else:
        cls = "NATURAL"

    return cls, ai, tamper

# ================= RISK SCORE =================
def fraud_risk_score(ai,tamper):
    score = ai*15 + tamper*12
    return min(score,100)

# ================= HEATMAP =================
def region_heatmap(path, grid=8):
    img = load_image(path)
    h,w,_ = img.shape
    heat = np.zeros((grid,grid))

    for i in range(grid):
        for j in range(grid):
            y1 = int(i*h/grid); y2 = int((i+1)*h/grid)
            x1 = int(j*w/grid); x2 = int((j+1)*w/grid)
            patch = img[y1:y2,x1:x2]
            res = extract_noise_residual(patch)
            heat[i,j] = np.std(res)

    heat = cv2.resize(heat,(w,h))
    return heat

# ================= REPORT GENERATOR =================
def generate_report(path,vector,cls,ai,tamper,risk):
    report = {
        "file": path,
        "timestamp": str(datetime.now()),
        "classification": cls,
        "ai_score": ai,
        "tamper_score": tamper,
        "fraud_risk": risk,
        "forensic_vector": vector
    }

    name = path.split('/')[-1].split('.')[0]
    with open(f"{name}_forensic_report.json","w") as f:
        json.dump(report,f,indent=4)

    return report

# ================= PIPELINE =================
def full_forensic_pipeline(path):
    img = load_image(path)
    vector = forensic_vector(img,path)
    cls, ai, tamper = multi_class_engine(vector)
    risk = fraud_risk_score(ai,tamper)
    heat = region_heatmap(path)
    report = generate_report(path,vector,cls,ai,tamper,risk)
    return vector, cls, ai, tamper, risk, heat, report

# ================= VISUAL =================
def visualize_all(path, heat):
    img = load_image(path)
    plt.figure(figsize=(12,4))

    plt.subplot(1,3,1)
    plt.title('Original')
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.title('AI/Tamper Heatmap')
    plt.imshow(heat, cmap='hot')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.title('Overlay')
    plt.imshow(img)
    plt.imshow(heat, cmap='hot', alpha=0.4)
    plt.axis('off')

    plt.show()

# ================= TECHNIQUE-WISE HEATMAPS =================
def heatmap_noise(path):
    img = load_image(path)
    res = extract_noise_residual(img)
    norm = cv2.normalize(np.abs(res), None, 0, 255, cv2.NORM_MINMAX)
    return norm


def heatmap_edges(path):
    img = load_image(path)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = sobel(gray)
    norm = cv2.normalize(edges, None, 0, 255, cv2.NORM_MINMAX)
    return norm


def heatmap_frequency(path):
    img = load_image(path)
    gray = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),(256,256))
    f = fftshift(fft2(gray))
    mag = np.log(np.abs(f)+1)
    mag = cv2.resize(mag,(img.shape[1],img.shape[0]))
    norm = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    return norm


def heatmap_entropy(path, grid=8):
    img = load_image(path)
    h,w,_ = img.shape
    heat = np.zeros((grid,grid))
    for i in range(grid):
        for j in range(grid):
            y1=int(i*h/grid); y2=int((i+1)*h/grid)
            x1=int(j*w/grid); x2=int((j+1)*w/grid)
            patch = img[y1:y2,x1:x2]
            heat[i,j] = entropy_score(patch)
    heat = cv2.resize(heat,(w,h))
    norm = cv2.normalize(heat, None, 0, 255, cv2.NORM_MINMAX)
    return norm


def heatmap_cfa(path, grid=8):
    img = load_image(path)
    h,w,_ = img.shape
    heat = np.zeros((grid,grid))
    for i in range(grid):
        for j in range(grid):
            y1=int(i*h/grid); y2=int((i+1)*h/grid)
            x1=int(j*w/grid); x2=int((j+1)*w/grid)
            patch = img[y1:y2,x1:x2]
            heat[i,j] = cfa_periodicity(patch)
    heat = cv2.resize(heat,(w,h))
    norm = cv2.normalize(heat, None, 0, 255, cv2.NORM_MINMAX)
    return norm


# ================= VISUALIZATION DASHBOARD =================
def visualize_all_heatmaps(path):
    img = load_image(path)

    h1 = heatmap_noise(path)
    h2 = heatmap_edges(path)
    h3 = heatmap_frequency(path)
    h4 = heatmap_entropy(path)
    h5 = heatmap_cfa(path)

    plt.figure(figsize=(15,10))

    plt.subplot(2,3,1)
    plt.title('Original')
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(2,3,2)
    plt.title('Noise Residual Heatmap')
    plt.imshow(h1, cmap='hot')
    plt.axis('off')

    plt.subplot(2,3,3)
    plt.title('Edge Inconsistency Heatmap')
    plt.imshow(h2, cmap='hot')
    plt.axis('off')

    plt.subplot(2,3,4)
    plt.title('Frequency Anomaly Heatmap')
    plt.imshow(h3, cmap='hot')
    plt.axis('off')

    plt.subplot(2,3,5)
    plt.title('Entropy Variation Heatmap')
    plt.imshow(h4, cmap='hot')
    plt.axis('off')

    plt.subplot(2,3,6)
    plt.title('CFA Physics Violation Heatmap')
    plt.imshow(h5, cmap='hot')
    plt.axis('off')

    plt.show()

print("FULL FORENSIC SYSTEM READY + TECHNIQUE HEATMAPS")


In [ ]:
path = "test_image.png"

visualize_all_heatmaps(path)


Batch_processing

In [ ]:
import os

data_dir = "data"

results = []

for file in os.listdir(data_dir):
    if file.lower().endswith((".jpg", ".jpeg", ".png")):
        path = os.path.join(data_dir, file)

        try:
            vector, cls, ai, tamper, risk, heat, report = full_forensic_pipeline(path)

            print("="*60)
            print("File:", file)
            print("Classification:", cls)
            print("AI Score:", ai)
            print("Tamper Score:", tamper)
            print("Fraud Risk:", risk)
            print("Report Generated:", f"{file.split('.')[0]}_forensic_report.json")

            results.append({
                "file": file,
                "classification": cls,
                "ai_score": ai,
                "tamper_score": tamper,
                "fraud_risk": risk
            })

        except Exception as e:
            print("❌ Error processing:", file, "->", str(e))

print("\n✅ Batch Processing Completed")
